*********************************
      Import Data + Packages
*********************************

In [1]:
#create working directory
import os#Miscellaneous operating system interfaces
os.chdir(r'C:\\Users\\rtreichl\\Documents\\competitions\\telstra')  #working directory

#%matplotlib inline allows graphics to show below each cell (or graphics in line)
# for some reason, %matplotlib inline won't work if comments are made in the same cell

In [2]:
%matplotlib inline

In [3]:
import pandas as pd #munging and wrangling
import numpy as np  #for arrays, etc.
import matplotlib.pyplot as plt #graphs/plots

#Regression Multi-Output
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor #aka as Extremely Randomized Trees

#Dimensionality Reduction 
from sklearn.decomposition import RandomizedPCA #to create eigenface

#Model Selection -> Cross Validation
from sklearn import cross_validation
from sklearn.cross_validation import cross_val_score

#Model Selection -> Metrics
from sklearn.metrics import mean_squared_error

#Model Selectin -> Parameter selection
from sklearn.grid_search import RandomizedSearchCV

In [4]:
#use dataframes for EDA
ftrain = pd.read_csv('training.csv')
ftest  = pd.read_csv('test.csv')
flookup= pd.read_csv('IdLookupTable.csv')

*********************************
EDA / Preprocessing / Data Preparation
*********************************

In [5]:
#drop -> column has no purpose
flookup= flookup.drop(['Location'],axis=1)

In [6]:
#train descriptive stats
train_describe=ftrain.describe()

In [7]:
#show specifically the count of instances/records by variable
#2140 output variables are without null values
train_describe.ix[0]

left_eye_center_x            7039
left_eye_center_y            7039
right_eye_center_x           7036
right_eye_center_y           7036
left_eye_inner_corner_x      2271
left_eye_inner_corner_y      2271
left_eye_outer_corner_x      2267
left_eye_outer_corner_y      2267
right_eye_inner_corner_x     2268
right_eye_inner_corner_y     2268
right_eye_outer_corner_x     2268
right_eye_outer_corner_y     2268
left_eyebrow_inner_end_x     2270
left_eyebrow_inner_end_y     2270
left_eyebrow_outer_end_x     2225
left_eyebrow_outer_end_y     2225
right_eyebrow_inner_end_x    2270
right_eyebrow_inner_end_y    2270
right_eyebrow_outer_end_x    2236
right_eyebrow_outer_end_y    2236
nose_tip_x                   7049
nose_tip_y                   7049
mouth_left_corner_x          2269
mouth_left_corner_y          2269
mouth_right_corner_x         2270
mouth_right_corner_y         2270
mouth_center_top_lip_x       2275
mouth_center_top_lip_y       2275
mouth_center_bottom_lip_x    7016
mouth_center_b

In [8]:
#delete instances where missing outputs
ftrain= ftrain.dropna()
print(len(ftrain))

2140


In [9]:
#create horizontal images and predictors
#double training set by flipping image horizontally
#point right to left (or vice versa)
# change output variable x-coordinates by (96 - x)

#this code works but does not improve predictive performance

train_temp=[]
train_temp0=[]
train_temp=pd.DataFrame(columns=ftrain.columns)

for i in range(len(ftrain)):
    image=ftrain['Image'].iloc[i]
    image=image.split()
    image=np.array(image).reshape(96,96) # reshape to 9216 x 9216
#    image=image.astype(float)  #float
    image=image[:,::-1]  #flip the image horizontally
    image=image.reshape(1,9216) #put back as 1d array
    image_str=""
    for zz in range(len(image[0])):
        image_str=image_str + str(image[0,zz]) + " "
    train_temp0=pd.DataFrame({
            'Image'                    : [image_str],
            'left_eye_center_x'        : [((96-ftrain['right_eye_center_x'].iloc[i]))],
            'left_eye_center_y'        : [((ftrain['right_eye_center_y'].iloc[i]))],
            'right_eye_center_x'       : [((96-ftrain['left_eye_center_x'].iloc[i]))],
            'right_eye_center_y'       : [((ftrain['left_eye_center_y'].iloc[i]))],
            'left_eye_inner_corner_x'  : [((96-ftrain['right_eye_inner_corner_x'].iloc[i]))],
            'left_eye_inner_corner_y'  : [((ftrain['right_eye_inner_corner_y'].iloc[i]))],
            'left_eye_outer_corner_x'  : [((96-ftrain['right_eye_outer_corner_x'].iloc[i]))],
            'left_eye_outer_corner_y'  : [((ftrain['right_eye_outer_corner_y'].iloc[i]))],
            'right_eye_inner_corner_x' : [((96-ftrain['left_eye_inner_corner_x'].iloc[i]))],
            'right_eye_inner_corner_y' : [((ftrain['left_eye_inner_corner_y'].iloc[i]))],
            'right_eye_outer_corner_x' : [((96-ftrain['left_eye_outer_corner_x'].iloc[i]))],
            'right_eye_outer_corner_y' : [((ftrain['left_eye_outer_corner_y'].iloc[i]))],
            'left_eyebrow_inner_end_x' : [((96-ftrain['right_eyebrow_inner_end_x'].iloc[i]))],
            'left_eyebrow_inner_end_y' : [((ftrain['right_eyebrow_inner_end_y'].iloc[i]))],
            'left_eyebrow_outer_end_x' : [((96-ftrain['right_eyebrow_outer_end_x'].iloc[i]))],
            'left_eyebrow_outer_end_y' : [((ftrain['right_eyebrow_outer_end_y'].iloc[i]))],
            'right_eyebrow_inner_end_x': [((96-ftrain['left_eyebrow_inner_end_x'].iloc[i]))],
            'right_eyebrow_inner_end_y': [((ftrain['left_eyebrow_inner_end_y'].iloc[i]))],
            'right_eyebrow_outer_end_x': [((96-ftrain['left_eyebrow_outer_end_x'].iloc[i]))],
            'right_eyebrow_outer_end_y': [((ftrain['left_eyebrow_outer_end_y'].iloc[i]))],
            'nose_tip_x'               : [((96-ftrain['nose_tip_x'].iloc[i]))],
            'nose_tip_y'               : [((ftrain['nose_tip_y'].iloc[i]))],
            'mouth_left_corner_x'      : [((96-ftrain['mouth_right_corner_x'].iloc[i]))],
            'mouth_left_corner_y'      : [((ftrain['mouth_right_corner_y'].iloc[i]))],
            'mouth_right_corner_x'     : [((96-ftrain['mouth_left_corner_x'].iloc[i]))],
            'mouth_right_corner_y'     : [((ftrain['mouth_left_corner_y'].iloc[i]))],
            'mouth_center_top_lip_x'   : [((96-ftrain['mouth_center_top_lip_x'].iloc[i]))],
            'mouth_center_top_lip_y'   : [((ftrain['mouth_center_top_lip_y'].iloc[i]))],
            'mouth_center_bottom_lip_x': [((96-ftrain['mouth_center_bottom_lip_x'].iloc[i]))],
            'mouth_center_bottom_lip_y': [((ftrain['mouth_center_bottom_lip_y'].iloc[i]))],
        })
    train_temp=train_temp.append(train_temp0)

print(len(train_temp))
print(train_temp.shape)

2140
(2140, 31)


In [10]:
#combine original train data with horizontal train data 
ftrain=ftrain.append(train_temp)
print(len(ftrain))
print(ftrain.shape)

4280
(4280, 31)


In [11]:
columntitles=['left_eye_center_x',
 'left_eye_center_y',
 'left_eye_inner_corner_x',
 'left_eye_inner_corner_y',
 'left_eye_outer_corner_x',
 'left_eye_outer_corner_y',
 'left_eyebrow_inner_end_x',
 'left_eyebrow_inner_end_y',
 'left_eyebrow_outer_end_x',
 'left_eyebrow_outer_end_y',
 'mouth_center_bottom_lip_x',
 'mouth_center_bottom_lip_y',
 'mouth_center_top_lip_x',
 'mouth_center_top_lip_y',
 'mouth_left_corner_x',
 'mouth_left_corner_y',
 'mouth_right_corner_x',
 'mouth_right_corner_y',
 'nose_tip_x',
 'nose_tip_y',
 'right_eye_center_x',
 'right_eye_center_y',
 'right_eye_inner_corner_x',
 'right_eye_inner_corner_y',
 'right_eye_outer_corner_x',
 'right_eye_outer_corner_y',
 'right_eyebrow_inner_end_x',
 'right_eyebrow_inner_end_y',
 'right_eyebrow_outer_end_x',
 'right_eyebrow_outer_end_y',
              'Image']
ftrain=ftrain.reindex(columns=columntitles)

In [12]:
#convert train-1 to array with 9216 width
x_train=ftrain['Image'].str.split()
x_train=np.vstack(x_train)
x_train=x_train.astype(float)
print(len(x_train))
print(x_train.shape)

4280
(4280, 9216)


In [13]:
#create output variable train-1
#create output variables
y_train=np.array(ftrain.ix[:,0:30])

print(len(y_train))
print(y_train.shape)

4280
(4280, 30)


In [14]:
#create test set
x_test=ftest['Image'].str.split()
x_test=np.vstack(x_test)
x_test=x_test.astype(float)
print(len(x_test))
print(x_test.shape)

1783
(1783, 9216)


*************************************
          Additional EDA
*************************************

In [ ]:
#further inspection of plots that outly the cluster show people with glasses
#'left_eye_center_x', 'left_eye_center_y'
#visualize predictor look for outliers
fig1=plt.figure()
ax=fig1.add_subplot(1,1,1)
ax.scatter(ftrain['left_eye_center_x'],ftrain['left_eye_center_y'],color="red")
ax.scatter(ftrain['right_eye_center_x'],ftrain['right_eye_center_y'],color="green")
ax.scatter(ftrain['mouth_center_top_lip_x'],ftrain['mouth_center_top_lip_y'],color="blue")
plt.ylim([0,96])
plt.xlim([0,96])
plt.show()

In [ ]:
#'left_eye_inner_corner_x'left_eye_inner_corner_y'
#visualize predictor look for outliers
fig1=plt.figure()
ax=fig1.add_subplot(1,1,1)
ax.scatter(ftrain['left_eye_inner_corner_x'],ftrain['left_eye_inner_corner_y'], color="red")
ax.scatter(ftrain['right_eye_inner_corner_x'],ftrain['right_eye_inner_corner_y'], color="green")
ax.scatter(ftrain['mouth_left_corner_x'],ftrain['mouth_left_corner_y'],color="blue")
plt.show()

In [ ]:
#'left_eyebrow_inner_end_x','left_eyebrow_inner_end_y'
#visualize predictor look for outliers
fig1=plt.figure()
ax=fig1.add_subplot(1,1,1)
ax.scatter(ftrain['left_eyebrow_inner_end_x'],ftrain['left_eyebrow_inner_end_y'],color="red")
ax.scatter(ftrain['right_eyebrow_inner_end_x'],ftrain['right_eyebrow_inner_end_y'],color="green")
ax.scatter(ftrain['mouth_center_bottom_lip_x'],ftrain['mouth_center_bottom_lip_y'],color="blue")
plt.show()

In [ ]:
# 'left_eye_outer_corner_x', 'left_eye_outer_corner_y', 
#visualize predictor look for outliers        
fig1=plt.figure()
ax=fig1.add_subplot(1,1,1)
ax.scatter(ftrain['left_eye_outer_corner_x'],ftrain['left_eye_outer_corner_y'],color="red")
ax.scatter(ftrain['right_eye_outer_corner_x'],ftrain['right_eye_outer_corner_y'],color="green")
ax.scatter(ftrain['mouth_right_corner_x'],ftrain['mouth_right_corner_y'], color="blue")
plt.show()

In [ ]:
#'left_eyebrow_outer_end_x','left_eyebrow_outer_end_y', 
#visualize predictor look for outliers
fig1=plt.figure()
ax=fig1.add_subplot(1,1,1)
ax.scatter(ftrain['left_eyebrow_outer_end_x'],ftrain['left_eyebrow_outer_end_y'],color="red")
ax.scatter(ftrain['right_eyebrow_outer_end_x'],ftrain['right_eyebrow_outer_end_y'],color="green")
ax.scatter(ftrain['nose_tip_x'],ftrain['nose_tip_y'], color="blue")
plt.show()

In [ ]:
#Example image before stretching
image_o=ftrain['Image'].iloc[0]
image_o=image_o.split()
image_o=np.array(image_o).reshape(96,96) # reshape to 96 x 96
image_o=image_o.astype(float)  #float
plt.matshow(image_o, fignum=100, cmap=plt.cm.gray)

In [ ]:
#Example Image after stretching
# Contrast stretching
p5, p95 = np.percentile(image_o, (5, 95))
img_rescale = exposure.rescale_intensity(image_o, in_range=(p5, p95))
plt.matshow(img_rescale, fignum=100, cmap=plt.cm.gray)

********************************
     Dimension Reduction
********************************

In [ ]:
################################
#    PCA
###############################

#pca
pca_estimator  = RandomizedPCA(n_components=1400,whiten=True)#whiten=True parameter make it possible to project the data onto the 
#singular space while scaling each component to unit variance. 

#pca train
x_train_pca = pca_estimator.fit_transform(x_train)#Fit the model with X and apply the dimensionality reduction on X.
print(x_train_pca.shape)

#pca test
x_test_pca  = pca_estimator.transform(x_test)#Apply dimensionality reduction on X.
print(x_test_pca.shape)

In [ ]:
#Identify Number of components to model
#given Kaggle is about finding the highest score (versus most useful), we will define the elbow as 32 components.
#However, given we are using Trees, it may not be useful to identify number of components.
ex_var=pca_estimator.explained_variance_ratio_#reduce to 150 to visually see the elbow
ex_var_all=pd.DataFrame(pca_estimator.explained_variance_ratio_)
ex_var_all.to_csv("ex_var_all.csv") 
plt.plot(ex_var)
plt.xlabel('Component')
plt.ylabel('Explained Variance in %')
plt.title('Explained Variance by Component')
plt.legend(loc='best')

***************************************
Create Training, Test, and holdout sets
***************************************

In [ ]:
#pca training and test sets for cross validation
x_traincv_pca, x_testcv_pca, y_traincv_pca, y_testcv_pca = cross_validation.train_test_split(x_train_pca, 
                                                                             y_train, test_size=0.4, random_state=0)

In [ ]:
#training and test sets for cross validation
x_traincv, x_testcv, y_traincv, y_testcv = cross_validation.train_test_split(x_train,y_train, test_size=0.4, random_state=0)

*****************************
Random Forest Regressor
*****************************

In [ ]:
###################################
#Random Forest Parameter Selection#
###################################
#function to 
def rf_params(xt,yt):
    param_grid ={"max_features": [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,
                                  30,31,32,33,34,35,36,37,38],
                 "max_depth" : [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,
                                32,33,34,35,36,38,39,40,41,42,43,44,45,46,47,48],
                 "min_samples_split": [1,2,3,4,5,6,7,8],
                 "min_samples_leaf" : [1,2,3,4,5,6,7,8],}

    rf=RandomForestRegressor(n_jobs=-1, oob_score=True)

    grid=RandomizedSearchCV(estimator=rf, param_distributions=param_grid,scoring="mean_squared_error", 
                            n_jobs=-1, cv=5,n_iter=1000)

    grid_train = grid.fit(xt,yt)

    print(grid.best_params_)


    
#rf_params(xt=x_train,yt=y_train)
rf_params(xt=x_testcv_pca,yt=y_testcv_pca)

#32 components {'min_samples_leaf': 2, 'max_features': 14, 'max_depth': 43, 'min_samples_split': 2}
#36 components {'min_samples_split': 4, 'max_features': 18, 'min_samples_leaf': 3, 'max_depth': 18}
#39'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 18, 'max_depth': 44}

In [16]:
#################################
#  RandomForestRegressor        #
#################################

def rf_model(label,x_train,y_train,x_test,# y_test,
             max_depth,max_features,min_samples_leaf,min_samples_split):

    rf_model = RandomForestRegressor(n_estimators=11119,n_jobs=-1,oob_score = True, max_depth=max_depth,max_features=max_features,
                                     min_samples_leaf=min_samples_leaf,min_samples_split=min_samples_split)
    rf_model_fit = rf_model.fit(x_train, y_train)
    y_pred_train = pd.DataFrame(rf_model_fit.predict(x_train))
    y_pred_test  = pd.DataFrame(rf_model_fit.predict(x_test))
    mse_train = mean_squared_error(y_train, y_pred_train) 
#    mse_test  = mean_squared_error(y_test , y_pred_test)
    print("train mse: " + str(mse_train))
#    print("test mse: " + str(mse_test))

    #create submission file
    location_df=[]
    location_df=pd.DataFrame(columns=['ImageId','FeatureName','Location'])
    FeatureName_list=ftrain.columns[0:30]
    i=0

    for FName in FeatureName_list:
        for z in range(len(ftest)):
            sub_temp=pd.DataFrame({'ImageId':[ftest['ImageId'].iloc[z]],
                              'FeatureName':[FName],
                              'Location':[y_pred_test.ix[z,i:i].values]})
            location_df=location_df.append(sub_temp)
        i=i+1

    Submission_df=pd.merge(flookup,location_df)
    Submission_df=Submission_df.drop(['ImageId'],axis=1)
    Submission_df=Submission_df.drop(['FeatureName'],axis=1)
    Submission_df['Location']=Submission_df['Location'].astype(float)
    Submission_df.to_csv("submission_" + label + ".csv", index=False)

In [17]:
#original data
rf_model(label="rf_over3",x_train=x_train,y_train=y_train,x_test=x_test,
         max_depth=50,max_features=2016,min_samples_leaf=1,min_samples_split=1)

#{'min_samples_leaf': 3, 'max_depth': 42, 'max_features': 46, 'min_samples_split': 5}
#{'max_depth': 39, 'max_features': 38, 'min_samples_split': 4, 'min_samples_leaf': 3}

#{'max_depth': 30, 'max_features': 28, 'min_samples_split': 2, 'min_samples_leaf': 2}

train mse: 0.696614577332


In [ ]:
#original data 60% training set
rf_model(label="rf_o_cv",x_train=x_traincv,y_train=y_traincv,x_test=x_testcv, #y_test=y_testcv,
         max_depth=42,max_features=46,min_samples_leaf=3,min_samples_split=5)

In [ ]:
#pca full training set
rf_model(label="rf_pca_4_31_3_18",x_train=x_train_pca,y_train=y_train,x_test=x_test_pca, #y_test=y_testcv,
         max_depth=31,max_features=38,min_samples_leaf=3,min_samples_split=4)

#{'min_samples_split': 4, 'max_depth': 31, 'min_samples_leaf': 3, 'max_features': 18}

In [ ]:
#pca cross validation
rf_model(label="rf_pca_cv",x_train=x_traincv_pca,y_train=y_traincv_pca,x_test=x_testcv_pca, y_test=y_testcv_pca,
         max_depth=31,max_features=38,min_samples_leaf=3,min_samples_split=4)

#{'min_samples_split': 4, 'max_depth': 31, 'min_samples_leaf': 3, 'max_features': 18}

In [ ]:
#training and test sets for cross validation
x_traincv, x_testcv, y_traincv, y_testcv = cross_validation.train_test_split(x_train,y_train, test_size=0.4, random_state=0)
for i in range(15,50):
    pca_estimator  = RandomizedPCA(n_components=i,whiten=True)
    x_train_pca = pca_estimator.fit_transform(x_traincv)
    x_test_pca  = pca_estimator.transform(x_testcv)
    rf_model = RandomForestRegressor(n_estimators=1016,n_jobs=-1,max_depth=44,max_features=i,
                                     min_samples_leaf=2,min_samples_split=2)
#{'min_samples_leaf': 2, 'max_features': 14, 'max_depth': 43, 'min_samples_split': 2}
#{'min_samples_split': 4, 'max_features': 18, 'min_samples_leaf': 3, 'max_depth': 18}
#'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 18, 'max_depth': 44}
#validation that 39 is optimal for number of components
    rf_model_fit = rf_model.fit(x_train_pca, y_traincv)
    y_pred_train = pd.DataFrame(rf_model_fit.predict(x_train_pca))
    y_pred_test  = pd.DataFrame(rf_model_fit.predict(x_test_pca))
    mse_train = mean_squared_error(y_traincv, y_pred_train)
    mse_test = mean_squared_error(y_testcv, y_pred_test)
    print("train mse: " + str(mse_train) + "test mse: "+ str(mse_test)+ "num comp: "+ str(i))    

****************************************
Extra Trees Regressor()
****************************************

In [ ]:
###################################
#Extra Trees   Parameter Selection#
###################################
#function to 
def et_params(xt,yt):
    param_grid ={"max_features": [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,
                                32,33,34,35,36,37,],
                 "max_depth" : [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,
                                32,33,34,35,36,38,39,40,41,42,43,44,45,46,47,48],
                 "min_samples_split": [1,2,3,4,5,6,7],
                 "min_samples_leaf" : [1,2,3,4,5,6,7],}

    rf=RandomForestRegressor()

    grid=RandomizedSearchCV(estimator=rf, param_distributions=param_grid,scoring="mean_squared_error", 
                            n_jobs=-1, cv=5,n_iter=1000)

    grid_train = grid.fit(xt,yt)

    print(grid.best_params_)

#et_params(xt=x_train,yt=y_train)
et_params(xt=x_train_pca,yt=y_train)

In [ ]:
############################
#  ExtraTreesRegressor     #
############################

et_model = ExtraTreesRegressor(n_estimators=4016,n_jobs=-1,max_depth=27, min_samples_split=3, 
                               max_features=18, min_samples_leaf=2)
#{'min_samples_leaf': 2, 'max_features': 18, 'max_depth': 27, 'min_samples_split': 3}

et_model_fit = et_model.fit(x_traincv_pca, y_traincv_pca)

y_pred_train = pd.DataFrame(et_model_fit.predict(x_traincv_pca))

y_pred_test  = pd.DataFrame(et_model_fit.predict(x_testcv_pca))

mse_train = mean_squared_error(y_traincv_pca, y_pred_train) 
mse_test = mean_squared_error(y_testcv_pca, y_pred_test) 

print("train mse: " + str(mse_train))
print("test mse: "  + str(mse_test))

location_df=pd.DataFrame(columns=['ImageId','FeatureName','Location'])
FeatureName_list=ftrain.columns[0:30]
i=0
for FName in FeatureName_list:
    for z in range(len(ftest)):
        sub_temp=pd.DataFrame({'ImageId':[ftest['ImageId'].iloc[z]],
                              'FeatureName':[FName],
                              'Location':[y_pred_test.ix[z,i:i].values]})
        location_df=location_df.append(sub_temp)
    i=i+1
    
Submission_df=pd.merge(flookup,location_df)
Submission_df=Submission_df.drop(['ImageId'],axis=1)
Submission_df=Submission_df.drop(['FeatureName'],axis=1)
Submission_df['Location']=Submission_df['Location'].astype(float)

Submission_df.to_csv("submission_et_pca_cv.csv", index=False)

******************************
          Decision Tree
******************************

In [ ]:
###################################
#Decision Tree Parameter Selection#
###################################
#function to 
def dt_params(xt,yt):
    param_grid ={"max_features": [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,
                                32,33,34,35,36,37,38],
                 "max_depth" : [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,
                                32,33,34,35,36,38,39,40,41,42,43,44,45,46,47,48],
                 "min_samples_split": [1,2,3,4,5,6,7,8,9,10,11,12],
                 "min_samples_leaf": [1,2,3,4,5,6,7,8,9,10,11,12],}

    rf=DecisionTreeRegressor()

    grid=RandomizedSearchCV(estimator=rf, param_distributions=param_grid,scoring="mean_squared_error", 
                            n_jobs=-1, cv=5,n_iter=1000)

    grid_train = grid.fit(xt,yt)

    print(grid.best_params_)

dt_params(xt=x_train,yt=y_train)

In [ ]:
dtr_model = DecisionTreeRegressor(max_features=18, min_samples_split=2,max_depth=44,min_samples_leaf=1)
#{'max_depth': 44, 'max_features': 38, 'min_samples_split': 4, 'min_samples_leaf': 12}

dtr_model_fit = dtr_model.fit(x_train, y_train)

y_pred_train = pd.DataFrame(dtr_model_fit.predict(x_train))

y_pred_test  = pd.DataFrame(dtr_model_fit.predict(x_test))

mse_train = mean_squared_error(y_train, y_pred_train)
print(mse_train)


location_df=pd.DataFrame(columns=['ImageId','FeatureName','Location'])
FeatureName_list=ftrain.columns[0:30]
i=0
for FName in FeatureName_list:
    for z in range(len(ftest)):
        sub_temp=pd.DataFrame({'ImageId':[ftest['ImageId'].iloc[z]],
                              'FeatureName':[FName],
                              'Location':[y_pred_test.ix[z,i:i].values]})
        location_df=location_df.append(sub_temp)
    i=i+1
    
Submission_df=pd.merge(flookup,location_df)
Submission_df=Submission_df.drop(['ImageId'],axis=1)
Submission_df=Submission_df.drop(['FeatureName'],axis=1)
Submission_df['Location']=Submission_df['Location'].astype(float)

Submission_df.to_csv("submission_dtreg_default.csv", index=False)